# Домашнее задание 1 - Вспомогательные задачи в СV



## Оценивание

Задание должно быть выполнено самостоятельно. Похожие решения будут считаться плагиатом. Если вы опирались на внешний источник в реализации, необходимо указать ссылку на него. 

В качестве решения, необходимо предоставить код + отчет, в котором будут отображены все детали выбора гиперпараметров, комментарии, сопровождающие графики, а так же ответы на вопросы в ДЗ. Оформляйте отчет четко и читаемо. Плохо оформленный код, плохо оформленные графики негативно скажутся на оценке, так же как и неэффективная реализация.


## Задание

Вам предстоит реализовать одну из вспомогательных задач &mdash; предсказание поворотов картинок (связанная статья - [Unsupervised Representation Learning by predicting Image Rotations](https://arxiv.org/pdf/1803.07728.pdf)). Возьмите датасет STL10, который прекрасно подходит под сеттинг предобучения без учителя. За архитектурное решение возьмите ResNet18. И датасет, и реализацию нейронки можно найти в библиотеке `torchvision` (`torchvision.datasets.STL10`, `torchvision.models.resnet.resnet18`). Для эффективности, используйте стандартные [оптимизации](https://pytorch.org/tutorials/recipes/recipes/tuning_guide.html) для обучения, например `Automatic Mixed Precision`. При желании, можно попробовать `nvidia.dali`.

**NB** По каждому из нижепредставленных пунктов, необходимо указать явно использованные для  моделей гиперпараметры и визуализации кривых обучения всех обучаемых моделей.

### [Датасет STL-10](https://cs.stanford.edu/~acoates/stl10/)

В STL10 всего 10 классов, а все изображения имеют разрешение $96 \times 96$ пикселей, что отличается от ImageNet (1000 классов и $224\times224$). Нужно адаптировать модель к картинкам меньшего размера и меньшему числу классов (подсказка: `model.conv1, model.maxpool, model.fc`).
Далее, будем называть энкодером или `backbone` часть модели до последнего слоя (`model.fc`). Репрезентациями - выход с `model.avgpool`.

Две части датасета подходят под стадию предобучения и стадию обучения на конечной задаче.
Одна из них (`unlabeled`) состоит из 100 000 изображений без разметки. 
Другая -  с разметкой, 5 000 изображений обучающей выборки (`train`) и 8 000 изображений валидационной выборки (`test`). Разметка состоит из 10 классов: `airplane`, `bird`, `car`, `cat`, `deer`, `dog`, `horse`, `monkey`, `ship`, `truck`, так что в размеченной выборке 500 и 800 изображений на каждый класс, соответственно.




## 0. Supervised 1
(0 баллов)

Для начала сделайте бейзлайн. Обучите резнет на размеченной части датасета без аугментаций, используя `train` и `test` в соотвествующих целях. Зафиксируйте лучшее достигнутое вами качество, ~70% top-1 accuracy будет засчитано. 

**NB** Без этого бейзлайна, домашнее задание не будет считаться выполненным (0 баллов за все).



## 1. Предсказание поворотов
(3 балла)

Вам необходимо обучить модель с нуля на предсказание угла поворота. В качестве обучающей и валидационной выборки используя `unlabeled`/`train` сплиты датасета STL10.

Определим допустимые повороты так - `[0, 90, 180, 270]` градусов, итого у нас 4 класса для задачи классификации (меняем `model.fc`), в которую мы превратили исходную. Теперь используя неразмеченную часть выборки (`split='unlabeled'`), необходимо сделать  из исходного датасета синтетический, который вместо пар (картинка, метка_класса) будет выдавать пары (картинка, метка_угла).
К примеру, можно определить новый класс `RotatedSTL10`, унаследовавшись от `VisionDataset`, а в нем реализовать случайный поворот на один из углов, и брать элементы так:

```python
def __getitem__(self, idx):
    img, _ = self.stl10_dataset[idx]
    img, angle = self.random_rotate(img)
    return img, angle
```

## 2. t-SNE
(1 балл)

Визуализируйте полученные векторные представления для `train` и `test` сплитов, используя t-SNE (`sklearn.manifold.TSNE`). Для этого необходимо брать репрезентации (`model.avgpool`), а также покрасить точки по классам (у каждого класса свой цвет). Хорошо ли разделяются исходные 10 классов?




## 3. Линейный пробинг
(1.5 балла)

Для того, чтобы сделать линейный пробинг предобученной модели, необходимо последний слой (`model.fc`) поменять на новый, подходящий под конечную задачу ('линейную пробу'). В этом случае &mdash; на родную для STL10 классификацию на 10 исходных классов. Заморозьте энкодер и обучите только добавленный слой на `train` сплите, провалидируйте на `test` сплите. Какого лучшего качества вам удалось добиться? Насколько оно лучше бейзлайна (обучения с учителем из случайной инициализации энкодера из пункта 0)? Сравните графики функции потерь бейзлайна и обучения линейной пробы.




## 4. Fine-tuning
(1.5 балла)

Попробуйте дообучить всю сеть полученную в пункте 1. Можно ли предложить более эффективную схему для файнтьюна? Какое качество вам удалось добиться здесь? Снова визуализируйте репрезентации с помощью t-SNE (как в пункте 2) и проанализируйте изменения.




## 5. Supervised 2
(1.5 балл)

Снова обучите модель с нуля на размеченной части датасета (обучение на `train`, валидация на `test`), на этот раз используя набор из стандартных аугментаций картинок (например, можно взять трансформации с семинара по аугментациям). Получается ли приблизиться к качеству (предобучения на поворотах + дообучения на исходной классификации)? Какая комбинация аугментаций дает лучшее качество?




## 6. Transfer Learning
(1.5 балл)

Возьмите предобученную на задаче предсказания поворота модель (из пункта 1). Обучите новую линейную пробу / дообучите модель полностью на датасете CIFAR-10. Какого качества получается добиться на новом датасете? Что это говорит об универсальности полученных репрезентаций?




**NB** **Бонусный** балл за красиво оформленный отчет.